<h1 style="color:red" align="center"> Projet d'introduction au Machine Learning <br/> Out-Of-Domain PoS Tagging <br/> Présentation des résultats algorithmique</h1>

In [1]:
#chargement des corpus en mémoire

import dataAnalysis as da
import pandas as pds
import matplotlib.pyplot as plt
import numpy as np
pds.set_option('display.max_colwidth', -1)
da.init()

Initialization...
	Preparing corpus fr.foot.test.json...
	Done
	Preparing corpus fr.gsd.test.json...
	Done
	Preparing corpus fr.sequoia.train.json...
	Done
	Preparing corpus fr.ftb.test.json...
	Done
	Preparing corpus fr.ftb.dev.json...
	Done
	Preparing corpus fr.spoken.test.json...
	Done
	Preparing corpus fr.pud.train.json...
	Done
	Preparing corpus fr.spoken.train.json...
	Done
	Preparing corpus fr.natdis.test.json...
	Done
	Preparing corpus fr.partut.dev.json...
	Done
	Preparing corpus fr.sequoia.test.json...
	Done
	Preparing corpus fr.gsd.dev.json...
	Done
	Preparing corpus fr.sequoia.dev.json...
	Done
	Preparing corpus fr.gsd.train.json...
	Done
	Preparing corpus fr.pud.test.json...
	Done
	Preparing corpus fr.ftb.train.json...
	Done
	Preparing corpus fr.partut.test.json...
	Done
	Preparing corpus fr.partut.train.json...
	Done
	Preparing corpus fr.spoken.dev.json...
	Done
Done!


# Statistique des corpus

In [ ]:
#itère sur chacun des corpus
def iterateCorpus(f, printing = True):
    for nameCorpus, corpus in da.listeCorpus.items():
        if printing:
            print("corpus : ", nameCorpus)
        for typeDS, dataset in corpus.getDataset().items():
            if printing:
                print("ensemble ", typeDS)
            f(dataset, nameCorpus, typeDS)

# met à jour les données statistiques
def updateStat(dataset, nm, tds):
    dataset.updateStat()

# affiche les statistiques basique


def printBasicStat(dataset, nm, tds):
    #baseStat.append((nm + " " + tds, len(dataset.data), dataset.nbWord, dataset.nbUniqueWords))
    print("   nombre de phrase : ", len(dataset.data))
    print("   nombre de mot : ", dataset.nbWord)
    print("   nombre de mot unique : ", dataset.nbUniqueWords)


    

# affiche les 10 mots / labels les plus fréquents de chaque corpus
def printMostFrequent(dataset, nm, tds):
    print()
    print("10 mots les plus fréquents : ")
    print(dataset.mostFrequentWord)
    print()
    print()
    print("labels les plus fréquents : ")
    print(dataset.mostFrequentLabel)
    print()
    
# affiche les 10 mots les plus ambigu de chaque corpus
def printMostAmbiguousWord(dataset, nm, tds):
    ambiguousDict, ambiguousWord = dataset.ambiguousWord()
    
    def sortSecond(val):
        return len(val[1])
    
    ambiguousWord.sort(key = sortSecond, reverse = True)
    
    print()
    print("5 mots les plus ambigu : ", ambiguousWord[:5])
    print()
    
iterateCorpus(updateStat, False)

In [ ]:
#baseStat = []

iterateCorpus(printBasicStat)

from tabulate import tabulate

#print(tabulate(baseStat, tablefmt="latex", floatfmt=".2f"))

In [ ]:
iterateCorpus(printMostFrequent)

In [ ]:
#mot les plus ambigu de chaque corpus

iterateCorpus(printMostAmbiguousWord)

## Out Of Vocabulary Word

In [ ]:
# Calcul des Out of Vocabulary Words
oovLatex = []


footTest = da.listeCorpus['foot'].testDataSet
natdisTest = da.listeCorpus['natdis'].testDataSet


for nameCorpus, corpus in da.listeCorpus.items():
    if corpus.trainExist:
        oovResult = corpus.computeCorpusOOV()
        oovResultFoot = corpus.computeOOV(footTest)
        oovResultnatdis = corpus.computeOOV(natdisTest)
        print(nameCorpus, " : ")
        print("     Pourcentage de l'oov entre train et test : ", oovResult[0])
        print("     Pourcentage de l'oov entre train et dev : ", oovResult[1])
        print("     Pourcentage de l'oov entre train et footTest : ", oovResultFoot)
        print("     Pourcentage de l'oov entre train et natdisTest : ", oovResultnatdis)
        latexRows = (nameCorpus, oovResult[0], oovResult[1], oovResultFoot, oovResultnatdis)
        oovLatex.append(latexRows)

print(tabulate(oovLatex, tablefmt="latex"))

## Divergence de Kullback-Leibler

In [ ]:
# Calcul des divergence de KullBack-Leibler

latexDKL = []

for nameCorpus, corpus in da.listeCorpus.items():
    if corpus.trainExist:
        print(nameCorpus + " : ")
        DKLResult = corpus.computeCorpusKLDivergence()
        DKLFootresult = corpus.computeKLDivergence(footTest)
        DKLnatdisResult = corpus.computeKLDivergence(natdisTest)
        print("     Dkl(test||train) = ", DKLResult)
        print("     Dkl(footTest||train) = ", DKLFootresult)
        print("     Dkl(natdisTest||train) = ", DKLnatdisResult)
        latexDKL.append( (nameCorpus, DKLResult, DKLFootresult, DKLnatdisResult) )
        
        
print(tabulate(latexDKL, tablefmt="latex"))

# Perplexity

In [ ]:
latexPerplexity = []

print("                         train,                   test,              dev")
for nameCorpus, corpus in da.listeCorpus.items():
    print(nameCorpus, " : ")
    pp_res = corpus.computePerplexityCorpus()
    print("     perplexity : ", pp_res)
    latexRow = []
    latexRow.append(nameCorpus)
    if type(pp_res) == tuple:
        for i in pp_res:
            latexRow.append(i)
    else:
        latexRow.append(pp_res)
    latexPerplexity.append( latexRow )
    
print(tabulate(latexPerplexity, tablefmt="latex"))

# Training Model
## Decision Tree Classifier

In [ ]:
import modelAnalysis as ma

In [ ]:
bestTree, bestParam = ma.analyzeDecisionTree("partut", maximum_depth=10, verbose=True)

In [ ]:
print(bestParam)

In [2]:
import features as ft
import DecisionTreeClassifier as dtree

trainSet, testSet = da.listeCorpus["partut"].trainDataSet, da.listeCorpus["partut"].testDataSet

Xtrain, Ytrain = ft.buildFeature(trainSet)
Xtest, Ytest = ft.buildFeature(testSet)

decisionTree = dtree.DecisionTreeClassifier(Xtrain, Ytrain, max_depth=16)

dt2 = dtree.DecisionTreeClassifier(Xtest, Ytest, max_depth=16)

print(np.shape(Xtrain))
print(np.shape(Ytrain))

(23324, 19)
(23324,)


In [ ]:
import time
t = time.time()
tree = decisionTree.fit()
print(time.time() - t)

In [ ]:
scoreTest, cmTest = decisionTree.modelScore(tree, Xtest, Ytest)

In [ ]:
print(scoreTest)

In [ ]:
ma.showConfusionMatrix(cmTest, decisionTree.classLabel.keys())

In [ ]:
decisionTreePartut.showTree(tree)

In [ ]:
import NaiveBayesClassifier as naivebc

naiveBC = naivebc.NaiveBayesClassifier(Xtrain, Ytrain)

In [ ]:
naiveBC.fit()

In [ ]:
score, cm = naiveBC.modelScore(Xtest, Ytest)

In [ ]:
print(score)

In [ ]:
ma.showConfusionMatrix(cm, decisionTree.classLabel.keys())

In [3]:
from sklearn.naive_bayes import MultinomialNB

In [5]:
Xtrain, Ytrain = ft.buildFeature(trainSet, lambda x:x)
Xtest, Ytest = ft.buildFeature(testSet, lambda x:x)


clf = MultinomialNB()
clf.fit(trainSet, testSet)

ValueError: Expected 2D array, got scalar array instead:
array=<dataAnalysis.DataSet object at 0x7ff151217438>.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
clf.score(Xtest, Ytest)